In [1]:
import pandas as pd
import numpy as np
import nlp_utils as nu

In [2]:
import pickle

In [3]:
tfid = pickle.load(open('tfidf.pickle','rb'))

In [4]:
# load the numpy zip
data = np.load('factors.npz')

In [5]:
# restoring array
factors = data['arr_0']

In [6]:
tfid

TfidfVectorizer()

In [7]:
df=pd.read_csv('Tf-Idf.csv',names=('Context','Response'))

In [8]:
df = df.astype(str)

In [9]:
df.head()

,Context,Response
0,reminder notification t emoticon t blank remin...,nan
1,reminder notification t emoticon,blank
2,remind me to buy a refill at time todau,for sure name what date would you like me to...
3,hi i want to set a reminder to drink water t b...,sure name i just need to know how frequently...
4,hi i want to set a reminder to drink water t b...,sure name i just need to know how frequently...


# Application

In [10]:
from sklearn.metrics.pairwise import cosine_distances

In [11]:
query = 'who are you ?'
def chatbot(query):
    # step:-1 clean
    query = nu.lemmatization_sentence(query)
    # step:-2 word embedding - transform
    query_vector = tfid.transform([query]).toarray()
    # step-3: cosine similarity
    similar_score = 1 -cosine_distances(factors,query_vector)
    index = similar_score.argmax() # take max index position
    # searching or matching question
    mathing_question = df.loc[index]['Context']
    response = df.loc[index]['Response']
    confidence = similar_score[index][0]
    chat_dict = {'match':mathing_question,
                'response':response,
                'score':confidence}
    return chat_dict

In [12]:
while True:
    query = input('USER: ')
    if query == 'exit':
        break
        
    response = chatbot(query)
    if response['score'] <= 0.2: # 
        print('Innomatics: Sorry!!! I can"t understand your question please rephrase your question')
    
    else:
        print('='*80)
        print('logs:\n Matched Question: %r\n Confidence Score: %0.2f'%(
            response['match'],response['score']*100))
        print('='*80)
        print('Innomatics: ',response['response'])

USER: hello
Innomatics: Sorry!!! I can"t understand your question please rephrase your question
USER: can you wake me up at 8 tomorrow morning
logs:
 Matched Question: 'wake me up at time morning'
 Confidence Score: 91.94
Innomatics:  for sure  name  would you like me to wake you up  date  or on some other date
USER: wake me on 23
logs:
 Matched Question: 'hi please wake me on time'
 Confidence Score: 69.44
Innomatics:  for sure  name  would you like me to wake you up  date  or on some other date
USER: yes wake me up on 21 january
logs:
 Matched Question: 'yes'
 Confidence Score: 69.75
Innomatics:  have a wonderful day ahead        sure  name  i just need to know how frequently i should remind you         chatform 65 
USER: remind me every 20 minutes
logs:
 Matched Question: 'hey can you send me regular reminder to drink water t sure name i just need to know how frequently i should remind you chatform 65 t every hour'
 Confidence Score: 32.39
Innomatics:   reminder interim          api